In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [3]:
ratings_data = pd.read_csv('./podaci/rating.csv', parse_dates=['timestamp'])

In [4]:
filtered_ratings_df = ratings_data[ratings_data['rating'] > 3]
print(filtered_ratings_df.shape)

(12195566, 4)


In [5]:
rand_userIds = np.random.choice(filtered_ratings_df['userId'].unique(), 
                                size=int(len(filtered_ratings_df['userId'].unique())*0.3), 
                                replace=False)

ratings = filtered_ratings_df.loc[filtered_ratings_df['userId'].isin(rand_userIds)]

print('Broj recenzija za treniranje: {} kreiranih od strane {} korisnika'.format(len(ratings), len(rand_userIds)))
ratings.sample(5)

Broj recenzija za treniranje: 3648172 kreiranih od strane 41508 korisnika


,userId,movieId,rating,timestamp
6902451,47556,4306,3.5,2006-12-10 05:03:19
19468739,134728,30707,3.5,2014-12-13 23:25:30
5493001,37764,3361,4.0,2001-02-23 07:21:39
18188288,125795,2075,4.5,2005-03-21 20:03:57
4797232,32957,1247,5.0,2000-12-11 16:11:40


In [209]:
# user_review_counts = filtered_ratings_df['userId'].value_counts()
# movie_review_counts = filtered_ratings_df['movieId'].value_counts()

# selected_users = user_review_counts[user_review_counts > 70].index
# selected_movies = movie_review_counts[movie_review_counts > 200].index

# filtered_ratings_df = filtered_ratings_df[(filtered_ratings_df['userId'].isin(selected_users)) & (filtered_ratings_df['movieId'].isin(selected_movies))]

In [6]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
ratings.loc[:, 'userId'] = user_encoder.fit_transform(ratings['userId'])
ratings.loc[:, 'movieId'] = movie_encoder.fit_transform(ratings['movieId'])
print(ratings.shape)
print(ratings.head())

(3648172, 4)
     userId  movieId  rating           timestamp
452       0       10     5.0 1996-12-25 15:29:11
455       0       61     5.0 1996-12-25 15:15:35
457       0      108     4.0 1996-12-25 15:26:09
459       0      139     5.0 1996-12-25 15:15:35
460       0      147     5.0 1996-12-25 15:25:14


In [7]:
#filtered_ratings = ratings.dropna(inplace=True)
#filtered_ratings = ratings.reset_index(drop=True)
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

In [21]:
user_input = tf.keras.layers.Input(shape=(1,))
user_embedding = tf.keras.layers.Embedding(input_dim=len(user_encoder.classes_), output_dim=50, input_length=1)(user_input)
user_flatten = tf.keras.layers.Flatten()(user_embedding)

movie_input = tf.keras.layers.Input(shape=(1,))
movie_embedding = tf.keras.layers.Embedding(input_dim=len(movie_encoder.classes_), output_dim=50, input_length=1)(movie_input)
movie_flatten = tf.keras.layers.Flatten()(movie_embedding)

concatenated = tf.keras.layers.Concatenate()([user_flatten, movie_flatten])

dense_layer = tf.keras.layers.Dense(128, activation='relu')(concatenated)
output_layer = tf.keras.layers.Dense(1, activation='linear')(dense_layer)

In [22]:
model = tf.keras.Model(inputs=[user_input, movie_input], outputs=output_layer)

In [23]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16066817329620481170
xla_global_id: -1
]


In [215]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_33 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 input_34 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 embedding_34 (Embedding)    (None, 1, 50)                2075400   ['input_33[0][0]']            
                                                                                                  
 embedding_35 (Embedding)    (None, 1, 50)                938300    ['input_34[0][0]']            
                                                                                           

In [8]:
train_user_data = train_data['userId'].values
train_movie_data = train_data['movieId'].values
train_ratings_data = train_data['rating'].values

test_user_data = test_data['userId'].values
test_movie_data = test_data['movieId'].values
test_ratings_data = test_data['rating'].values

In [217]:
model.fit([train_user_data, train_movie_data], train_ratings_data, epochs=10, batch_size=64, validation_data=([test_user_data, test_movie_data], test_ratings_data))

Epoch 1/10
45695/45695 [==============================] - 810s 18ms/step - loss: 0.2416 - mae: 0.3944 - val_loss: 0.2053 - val_mae: 0.3778
Epoch 2/10
45695/45695 [==============================] - 809s 18ms/step - loss: 0.2005 - mae: 0.3712 - val_loss: 0.2010 - val_mae: 0.3704
Epoch 3/10
45695/45695 [==============================] - 787s 17ms/step - loss: 0.1955 - mae: 0.3648 - val_loss: 0.2000 - val_mae: 0.3700
Epoch 4/10
45695/45695 [==============================] - 786s 17ms/step - loss: 0.1914 - mae: 0.3595 - val_loss: 0.1985 - val_mae: 0.3689
Epoch 5/10
45695/45695 [==============================] - 787s 17ms/step - loss: 0.1870 - mae: 0.3539 - val_loss: 0.1978 - val_mae: 0.3656
Epoch 6/10
45695/45695 [==============================] - 788s 17ms/step - loss: 0.1825 - mae: 0.3479 - val_loss: 0.1992 - val_mae: 0.3670
Epoch 7/10
45695/45695 [==============================] - 790s 17ms/step - loss: 0.1780 - mae: 0.3419 - val_loss: 0.2016 - val_mae: 0.3679
Epoch 8/10
45695/45695 [===

In [219]:
model.save('./modeli/model_1.h5')
model.save('./modeli/model_1.keras')

c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
loaded_model = tf.keras.models.load_model('./modeli/model_1.keras')

In [11]:
predictions = loaded_model.predict([test_user_data, test_movie_data])

22802/22802 [==============================] - 21s 926us/step


In [12]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_ratings_data, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.33967520709132004


In [13]:
score = loaded_model.evaluate([test_user_data, test_movie_data], test_ratings_data, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Test loss: 0.33967527747154236
Test accuracy: 0.4813114106655121
